In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
"""
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}

var colab = setInterval(ConnectButton,60000);
"""

In [ ]:
ROOT_DIR = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/"

# YOLO-MEDIUM

In [ ]:
%%time
import os

from ultralytics import YOLO

save_dir = os.path.join(ROOT_DIR,"yolo-medium")
# Load a model
model = YOLO("yolov8m.yaml")
model = YOLO("yolov8m.pt")

# train the model
results = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=150,imgsz=640,project = save_dir)

# YOLO SMALL

In [ ]:
%%time
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8s.yaml")
model = YOLO("yolov8s.pt")

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=100,imgsz=640)  # train the model

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!scp -r /content/runs "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/yolo-small"

In [ ]:
# results2 = model.train(resume=True)

In [ ]:
# model = YOLO("yolov8n.yaml")
# model = YOLO("/content/runs/detect/train/weights/last.pt")

# # Use the model
# results2 = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=2,imgsz=640)

In [ ]:
# model = YOLO("yolov8n.yaml")
# model = YOLO("/content/runs/detect/train2/weights/last.pt")

# # Use the model
# results2 = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=8,imgsz=640)

#Prediction

##Images

In [ ]:
import os

from ultralytics import YOLO
import cv2
medium_model_path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/yolo-medium/train/weights/best.pt"
small_model_path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/yolo-small/runs/detect/train/weights/best.pt"

#model_path = "/content/runs/detect/train/weights/best.pt"
model = YOLO(medium_model_path)

In [ ]:
import cv2

def write_label_bounding_box(img, class_id, x1, y1, x2, y2, score,result):
  score_str = 'Score: {:.2f}'.format(score)
  class_name = result.names[int(class_id)].replace("₹", "")
  text = class_name + ' ' + score_str

  if class_id == 0:
      color = (255, 128, 0)
  elif class_id == 1:
      color = (0, 165, 255)
  elif class_id == 2:
      color = (147, 20, 255)
  elif class_id == 3:
      color = (255, 0, 255)
  else:
      color = (0, 0, 0)  # Default color

  cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 20)
  cv2.putText(img, text, (int(x1), int(y1 - 30)), cv2.FONT_HERSHEY_SIMPLEX, 4, color, 20, cv2.LINE_AA)

  return img

In [ ]:
def prediction(img,model):
  #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
  results = model(img)#[0]
  result = results[0]
  threshold = 65
  output = {}
  output['₹1'],output['₹2'],output['₹5'],output['₹10'] = 0,0,0,0

  for i in result.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = i
    if score >= threshold/100:
      pred_class = result.names[class_id]

      output[pred_class] += 1
      img = write_label_bounding_box(img,class_id,x1, y1, x2, y2,score,result)


  total = (output['₹1'])+(2*output['₹2'])+(5*output['₹5'])+(10*output['₹10'])

  text = f"Total = {total}"
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 1
  color = (0, 255, 0)
  thickness = 2
  #text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
  height, width, _ = img.shape
  #print(height,width)

  # Define the position for the text (adjust as needed)
  x = height - 1000  # X-coordinate (vertical)
  y = width - 1000   # Y-coordinate (horizontal)
  x,y = 500,300

  # Put text on the image
  cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 10, color, 30, cv2.LINE_AA)

  return img

In [ ]:
path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/test-data /IMG20230827213305.jpg"
img = cv2.imread(path)
img = prediction(img,model)
scale_percent = 10 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)

# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
cv2_imshow(resized)

##Videos

In [ ]:
import cv2

def write_label_bounding_box(img, class_id, x1, y1, x2, y2, score,result):
  score_str = 'Score: {:.2f}'.format(score)
  class_name = result.names[int(class_id)].replace("₹", "")
  text = class_name + ' ' + score_str

  if class_id == 0:
      color = (255, 128, 0)
  elif class_id == 1:
      color = (0, 165, 255)
  elif class_id == 2:
      color = (147, 20, 255)
  elif class_id == 3:
      color = (255, 0, 255)
  else:
      color = (0, 0, 0)  # Default color

  cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
  cv2.putText(img, text, (int(x1), int(y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

  return img

In [ ]:
def prediction(img,model):
  #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
  results = model(img)#[0]
  result = results[0]
  threshold = 65
  output = {}
  output['₹1'],output['₹2'],output['₹5'],output['₹10'] = 0,0,0,0

  for i in result.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = i
    if score >= threshold/100:
      pred_class = result.names[class_id]

      output[pred_class] += 1
      img = write_label_bounding_box(img,class_id,x1, y1, x2, y2,score,result)


  total = (output['₹1'])+(2*output['₹2'])+(5*output['₹5'])+(10*output['₹10'])

  text = f"Total = {total}"
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 1
  color = (0, 255, 0)
  thickness = 2
  #text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
  height, width, _ = img.shape
  #print(height,width)

  # Define the position for the text (adjust as needed)
  x = height - 1000  # X-coordinate (vertical)
  y = width - 1000   # Y-coordinate (horizontal)
  x,y = 50,30

  # Put text on the image
  cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 4, cv2.LINE_AA)

  return img

In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Open the video file
video_path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/test-data /VID20230916140154.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = 'output_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        #results = model(frame)

        # Visualize the results on the frame
        #annotated_frame = results[0].plot()
        annotated_frame = prediction(frame,model)
        #cv2_imshow(annotated_frame)
        out.write(annotated_frame)
        # Convert annotated_frame to OpenCV format
        #annotated_frame = annotated_frame[:, :, ::-1]

        # Display the annotated frame
        #cv2_imshow(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
# Load the YOLOv8 model
#model = YOLO('yolov8n.pt')

# Open the video file
video_path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/test-data /VID20230916140100.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2_imshow(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

In [ ]:
path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/test-data /VID20230916140100.mp4"
#path = "/content/238.jpg"
results = model(path)#[0]
result = results[0]
result

In [ ]:
len(result.boxes)

In [ ]:
box = result.boxes[0]
cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
result.names

In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow
img = cv.imread(path, cv.IMREAD_UNCHANGED)

threshold = 65
output = {}
output['₹1'],output['₹2'],output['₹5'],output['₹10'] = 0,0,0,0

for i in result.boxes.data.tolist():
  x1, y1, x2, y2, score, class_id = i
  if score >= threshold/100:
    print(score,class_id)
    pred_class = result.names[class_id]

    output[pred_class] += 1
    # cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 20)
    # cv2.putText(img, result.names[int(class_id)].replace("₹",""), (int(x1), int(y1 - 10)),
    #             cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 20, cv2.LINE_AA)
    img = write_label_bounding_box(img,class_id,x1, y1, x2, y2,score)


total = (output['₹1'])+(2*output['₹2'])+(5*output['₹5'])+(10*output['₹10'])
print(f"Total is -------->{total}")
#img = cv2.imread('0.jpg')
scale_percent = 10 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)

# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
#cv2_imshow(resized)
#cv2.imwrite('0.jpg',img)

In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Open the video file
video_path = "/content/gdrive/MyDrive/yolov8/Object-Detection/coin-det/test-data /VID20230916140100.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = 'output_video.avi'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        out.write(annotated_frame)
        # Convert annotated_frame to OpenCV format
        #annotated_frame = annotated_frame[:, :, ::-1]

        # Display the annotated frame
        #cv2_imshow(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()